# ImageNet Classification with Deep Convolutional Neural Networks

> http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf

- __ 발표자 : 정지원__
- __ 발표일 : 2017. 8. 7(월)__
---

## Abstract

>ImageNet LSVRC-2010 contest

> - top-1 error/top-5 error > 37.5%, 17.0%

>ILSVRC-2012 __AlexNet__

> - top-1 error/top-5 error > __15.3%, 26.2%__

## Introduction

Object recognition

- collect larger datasets
- learn more powerful models
- better techniques for preventing overfitting

최근 datasets들은 아주 크다.

수 백만장의 images, 1,000개 이상의 objects -> need large learning capacity.

CNN은 이러한 것에 강력하다.

깊이와 폭을 다양하게 조절할 수 있으며, 이미지의 성질에 대해 강하다.

따라서 유사한 크기의 layer를 가진 standard feedforward NN과 비교하여 CNN은 Connection 및 parameter가 훨씬 적기 때문에 훈련하기가 더 쉽다.

이론적으로는 최상의 성능은 약간 더 안 좋을수도 있다.

CNN의 매력적인 특성과 로컬 아키텍처의 상대적 효율성에도 불구하고 많은 고해상도 이미지를 분류하는 것은 큰 비용이 들었다.

다행히 현재의 GPU는 2D Convolution에 매우 최적화된 구현을 통해 대규모 CNN의 교육이 쉽다.

ImageNet과 같은 data는 overfitting없이 모델을 학습하는데 충분한 labeled example을 갖고 있다.

ILSVRC-2010 및 ILSVRC-2012 대회에서 사용 된 ImageNet 데이터에 대해 최대 규모의 CNN를 훈련했고, 최고의 결과를 냈다.

GPU 구조 및 CNN 구조를 모두 공개했다.

성능을 향상시키고, 시간을 단축시킨 몇 가지 방법들은 뒤에서 소개한다.

5개의 Conv layer와 3개의 fully connected layer로 구성이 되어있으며, 오버피팅을 방지하기 위한 방법도 소개한다.

네트워크의 크기는 GPU 메모리와 허용되는 시간에서의 최대 크기로 구성된다.

GTX 580 3GB GPU 2대로 5~6일의 훈련이 걸렸다. 더 빠른 GPU가 존재한다면 결과를 향상시킬 수 있을 것이다.

## The Dataset

ImageNet은 22,000 category, 1,500만개 이상의 고해상도 이미지 데이터 세트다.

이미지는 웹에서 수집되어 Amarson의 Mechanical Turk 도구를 사용하여 사람에 의해 분류됐다.

2010년부터는 Pascal Visual Object Challenge의 일환으로 ILSVRC(ImageNet Large Scale Visual Recognition Challenge)라는 대회가 개최됐다.

ILSVRC는 1,000개의 범주에 약 1,000개의 이미지가 있는 ImageNet의 일부를 사용한다.

training 1,200,000/validation 50,000/test 150,000

top-1 / top-5 방식으로 평가

고정된 입력 사이즈가 필요, 이미지를 256 x 256의 고정 해상도로 다운 샘플링

직사각형 이미지가 주어지면 짧은 쪽의 길이가 256이되도록 이미지의 크기를 재조정한 다음, 결과의 중앙 256×256을 잘라 냈다.

각 픽셀에서 트레이닝 세트에 대한 __Subtracting Mean Activity__를 제외하고는 다른 방법으로 이미지를 전처리를 하지 않았다.

따라서, 네트워크의 centered RGB 값을 훈련 시켰다.

## The Architecture

전체적으로 8개의 layers로 구성

5개의 conv layers / 3개의 fully-connected layers

사용한 방법들에 대해 설명한다.

### ReLU Nonlinearity

![Image](figures/1.png)

기존의 activation function은 logistic(sigmoid)/tanh 였다.

위 방법들은 Graidnet descent가 있는 학습에서는, ReLU보다 훨씬 느리게 된다.

비선형성을 가진 뉴런을 Rectified Linear Units(ReLUs)라고 부른다.

ReLU를 사용한 deep CNN은 tanh 보다 몇 배 더 빠르게 훈련된다.

위 그래프는 CIFAR-10 데이터 세트에서 CNN의 특정 layer의 25%의 training error에 도달하는 데 필요한 반복 횟수를 보여준다.

전통적인 방법으로는 거의 불가능함을 알 수 있다.

빠른 학습은 큰 데이터셋에서 학습된 커다란 모델의 성능에 많은 영향을 미친다.

### Training on Multiple GPUs

하나의 GPU로는 학습할 수 있는 네트워크 최대 크기가 제한된다.

120만 개의 training data를 하나의 GPU에 맞추기는 힘들다.

분산처리를 이용

현재 GPU는 GPU병렬 처리에 특히 적합한다.

병렬화는 커널(또는 뉴런)의 절반을 각 GPU에 넣고 트릭을 이용했다.

GPU는 특정 계층에서만 통신하도록 구성했다.

하나의 GPU를 사용한 CNN과 비교할 때 top-1/top-5 error를  1.7%/1.2% 줄였다.

2-GPU net은 one-GPU net보다 학습시간이 짧다.

### Local Response Normalization

ReLU는 saturated 되지 않기위한 입력 normailzation을 필요로 하지 않는 좋은 특성을 가지고 있다.

적어도 일부 훈련 예제가 ReLU에 positive 입력을 발생시키는 경우 학습이 된다.

그러나, 우리는 여전히 다음의 local normalization 체계가 일반화를 돕는다는 것을 알 수 있다.

$a^{i}_{x,y}$는 커널 i를 위치(x, y)에 적용한 다음 ReLU 비선형성을 적용하여 계산된 뉴런의 활동을 나타낼 때,

Response normalized된 activity $b^{i}_{x,y}$는 합계가 _n_ "adjacent(인접)" 커널은 동일한 공간 위치에서 맵핑하며,

N은 계층에 있는 커널의 총 수다.

![Image](figures/2.png)

커널 맵의 순서는 물론 학습이 시작되기 전에 임의로 결정된다.

This sort of response normalization implements a form of lateral inhibition inspired by the type found in real neurons, creating competition for big activities amongst neuron outputs computed using different kernels.

상수 k, n, α 및 β는 값이 유효성 검증 세트를 사용하여 결정되는 Hyper-parameter다. 우리는 k=2, n=5, α=10-4 및 β=0.75를 사용했다.

특정 레이어에서 ReLU 비선형성을 적용한 후에 이 정규화를 적용했다.

이 기법은 Jarrett 등의 local contrast normalization 기법과 비슷하다.

우리는 평균 활동을 빼지 않기 때문에 더 정확하게 "Brightness normalization"라고 부른다.

Response Normalization는 우리의 top-1 및 top-5 오류율을 각각 1.4%/1.2% 감소시킨다.

우리는 또한 CIFAR-10 데이터 세트에서 이 스킴의 유효성을 확인했다. 4-layer CNN은 정규화가 없는 경우 13%의 테스트 오류율을, 

정규화를 사용하는 경우 11%의 test error-rate을 보인다.

### Overlapping Pooling

CNN의 pooling layer는 동일한 커널 맵에서 인접한 뉴런 그룹의 출력을 요약한다.

전통적으로, 인접한 풀링 유닛에 의해 요약 된 이웃은 중첩되지 않는다. (예를 들어, [17, 11, 4]).

보다 정확하게 말하면, 풀링 층은 풀링 유닛의 위치를 ​​중심으로하는 크기 z × z의 이웃을 요약하는, 각각 이격 된 풀링 유닛의 그리드로 구성되는 것으로 생각할 수있다.

s=z로 설정하면 CNN에서 일반적으로 사용되는 전통적인 로컬 풀링을 얻습니다.

s < z로 설정하면 Overlap pool이 된다.

이것은 네트워크 전체에서 s=2 및 z=3을 사용하는 것이다.

이 구성표는 비중첩 구성표와 비교할 때 top-1 및 top-5 오류율을 각각 0.4%/0.3% 줄인다.

s=2, z=2, 등가 치수의 출력을 생성한다.

우리는 일반적으로 Overlapping Pooling 된 모델이 overfitting을 방지하는데 더 좋다는 것을 확인했다.

### Overall Architecture

이제 CNN의 전반적인 아키텍처를 설명 할 준비가 됐다.

그림처럼 network은 8개의 layer로 구성된다.

처음 5개는 conv-layer고 나머지 3개는 fully-connected다.

output layer는 1,000개로 퍼지는 softmax로 구성된다.

네트워크는 다차원 logistic regression objective를 최대화한다.

이는 prediction distribution에서 올바른 label의 log-probability에 대한 training case의 평균을 최대화하는 것과 같다.

![Image](figures/3.png)

2, 4, 5 conv-layer의 kernel은 동일한 GPU에 있는 이전 layer의 커널 맵에만 연결된다.

3 번째 conv-layer의 커널은 두 번째 layer의 모든 커널 맵에 연결된다.

fully-connected layer의 뉴런은 이전 layer의 모든 뉴런에 연결된다.

Response Normailzation은 첫 번째 및 두 번째 conv-layer를 따른다. 

Max-pooling layer은 Response Normailzation layer와 다섯 번째 conv-layer을 따른다.

ReLU는 모든 conv-layer와 fully-connected layer의 output의 acitvation function으로 사용된다.

첫 번째 conv-layer는 224x224x3 입력 이미지를 4x4 픽셀의 크기로 11x11x3 크기의 96개 커널로 filtering 한다.

두 번째 conv-layer는 첫 번째 conv-layer의(RN되고 pooling 된) 출력을 입력으로 받아 5×5x48 크기의 256개의 커널로 filtering한다.

3, 4, 5 Conv-layer는 중간에 있는 pooling 또는 Normalization layer 세 번째 컨볼 루션 계층은 3x3x256 크기의 384개 커널을 두 번째 conv-layer의 RN-pooling된 출력에 연결한다.

네 번째 layer는 크기가 3×3×192인 384개의 커널을 가지며,

다섯 번째 conv-layer는 크기가 3×3×192인 256개의 커널을 포함한다.

fully-connected layer에는 각각 4096 개의 뉴런이 있다.

## Reducing Overfitting

6,000만개의 parameter

ILSVRC의 1,000개 class가 각 training image를 label로 mapping하는데 10비트의 제약을 부과 했음에도 불구하고,

overfitting없이 많은 파라미터를 학습할 수는 없다.

아래에서 우리는 overfitting과 싸우는 두 가지 기본 방법을 설명한다.

### Data Augmentation

이미지 데이터의 초과 맞춤을 줄이는 가장 쉽고 가장 일반적인 방법은 label 보존 변형을 사용하여 인위적으로 데이터 세트를 확대하는 것이다.

우리는 두 가지 형태의 데이터 확대 방법을 사용한다.

두 가지 방법 모두 매우 적은 계산량으로 원본 이미지에서 변환 된 이미지를 생성 할 수 있으므로 변환 된 이미지를 디스크에 저장할 필요가 없습니다.

구현시 변형된 이미지는 CPU의 Python 코드로 생성되는 반면 GPU는 이전 이미지 배치를 교육합니다.

따라서 이러한 데이터 증가 체계는 사실상 계산상 자유롭다.

- #### 이미지 translations과 horizontal reflection을 생성

> 256 × 256 이미지에서 임의의 224 × 224 패치(및 그 horizontal reflection)를 추출하여 이러한 추출 된 patches로 학습

> Training set의 크기를 2048 배 증가

> 이런 방법을 사용하지 않는다면, overfitting 때문에 망을 깊히 만드는게 불가능

> 테스트 시간에 네트워크는 224×224 패치 (4개의 corner patches와 centered patch)와 horizontal reflection(총, 10개)를 추출

> 이러한 10개의 patches로 네트워크의 softmax layer에 의해 만들어진 값을 평균하여 예측을 한다.

- #### Training 이미지에서 RGB 채널의 강도를 변경

> ImageNet training 세트에서 RGB 픽셀 값 집합에 대해 PCA를 수행

> 각 training 이미지에 대해, 발견된 주성분의 값을, 그 eigen-value와 $\mu$=0, $\sigma=0.1$를 따르는 Random value와의 곱을 더한다.

> 따라서, 각 RGB 이미지 픽셀에 $I_{xy} = [I^{R}_{xy}, I^{G}_{xy}, I^{B}_{xy}]^{T}$ 다음을 추가한다.

> ![Image](figures/9.png)

> $p_i$와 $\lambda_i$는 3x3 RGB pixel value에 대한 covariance matrix의 eigenvector, eignevalue다.

> $\alpha_i$는 random값이며, 한 번의 학습이 일어나는 동안 같은 값을 갖다가, 다음 번 학습때 새로 만들어진다.

> 이 방식으로 원본 이미지의 중요한 속성을 캡처합니다.

> 즉, 객체의 정체성은 조명의 강도와 색상의 변화에도 크게 변하지 않는다.

> 이 방식은 top-1 error rate을 1% 이상 줄인다.

### Dropout

Drop-out

0.5 확률로 output을 0으로 보낸다.

이런 식으로 "drop-out"된 뉴런은 feedforward training에 기여하지 않으며 backprop에도 참여하지 않는다.

따라서 매 input마다 NN은 다른 아키텍처를 sampling하지만 이러한 모든 아키텍처는 가중치를 공유한다.

이 테크닉은 뉴런이 특정 다른 뉴런의 존재에 의존할 수 없으므로 복잡한 뉴런의 co-adatation을 감소시킨다.

그러므로 많은 다른 부분 뉴런과 함께 유용하고 강력한 기능을 배우게 된다.

Test를 할 때 우리는 모든 뉴런을 사용하면서 0.5를 곱한다.

누락된 네트워크에 의해 생성된 예측 분포의 기하 평균을 취하는 것에 대한 합리적인 방법이다.

처음 fully-connected layer에서 drop-out을 사용한다.

Overfitting을 방지해준다.

Drop-out 수렴하는 데 필요한 반복 횟수를 대략 두 배로 늘리게 된다.

## Details of learning

Batch size 128, Momentum 0.9, weight decay 0.0005의 SGD을 사용하여 모델을 훈련 시켰다.

여기서 weight decaying의 중요함을 알 수 있었다.

즉, decaying은 단지 정규화가 아니라 모델의 학습 오류를 줄인다.

![Image](figures/5.png)

각 층의 weight는 표준 편차 0.01인 zero-mean gaussian 분포로부터 초기화시켰다.

우리는 2, 4, 5 번째 conv-layers뿐만 아니라 fully-connected layers의 bias를 상수 1로 초기화했다.

이 초기화는 ReLU에 양수 입력을 제공하여 학습의 초기 단계를 가속화한다.

나머지 layer의 bias는 상수 0으로 초기화했다.

모든 layers에 대해 동일한 학습 속도를 사용했는데, 학습을 통해 수동으로 조정되도록 했다.

Heuristic으로 Validation 오류율이 현재의 학습률로 개선되지 않을 때 학습률을 10으로 나누는 방식을 사용했다.

학습 속도는 0.01으로 초기화됐고 종료 전까지 3번 감소됐다.

두 개의 NVIDIA GTX 580 3GB GPU에서 5일에서 6일 걸린 120만 개의 이미지 셋을 통해 대략 90 cycle동안 네트워크를 학습시켰다.

## Results

ILSVRC-2010에 대한 우리의 결과는 다음과 같다.

37.5% / 17.0%의 top-1/top-5 error rate.

ILSVRC-2010 대회의 최고의 성과는 47.1%와 28.2%였는데,

다른 features로 학습된 6개의 Sparse-coding model에서 생성된 예측을 평균한 접근법이 있었고,

그 이후 가장 우수한 결과로는, 2가지 유형의 조밀하게 샘플링된 피쳐로부터 계산된 피셔 벡터(Fisher Vector, Fisher)에 대해

학습된 분류기의 예측을 평균화하는 접근법으로 45.7%/25.7%의 정확도를 보였었다.

![Image](figures/6.png)

ILSVRC-2010 대회 기간 동안 달성 된 최고의 성과는 47.1%와 28.2% 였는데,

이는 6개의 Sparse-coding model에서 얻은 예측을 평균화하는 접근 방식을 사용했다.

16.6 %의 2011년 가을에 발표된 5개의 CNN에 대해 pre-trained된 2개의 CNN의 예측을 평균하면 오류율은 15.3%다.

2nd best test entry는 다른 유형의 조밀하게 샘플링 된 피쳐로부터 계산된 FV에 대해 훈련 된 몇 가지 분류 자의 예측을 평균하는 접근법을 사용하여 26.2 %의 오류율을 달성했다.

그 이후 가장 우수한 결과는 두 가지 densely-sampled feature로부터 계산된 피셔 벡터 (Fisher Vector, FV)에 대해 훈련된 두 개의 classifier의 예측을 평균하는 접근법으로 45.7%와 25.7%다.

![Image](figures/7.png)

### Qualitative Evaluations

![Image](figures/4.png)

두 개의 GPU의 커널을 보자.

GPU 1의 커널은 색상에 영향을 받지 않지만, GPU 2의 커널은 색상에 따라 크게 다르다.

이러한 종류의 specialization는 모든 실행중에 발생하며 임의의 특정 가중치 초기화와는 독립적입니다.

![Image](figures/8.png)

왼쪽 상단의 벌레와 같이 중심에서 벗어난 물체도 network에 의해 인식될 수 있다.

상위 5개 label의 대부분은 합리적으로 보인다.

예를 들어, 다른 유형의 고양이들이 표범에 대한 그럴싸한 label로 나타난다.

(Grille, Cherry) 사진처럼 모호한 것들도 있다.

네트워크의 시각적 지식을 조사하는 또 다른 방법은 마지막 4,096차원 layer에서 이미지로 유도된 feature activations를 고려하는 것입니다.

두 이미지가 Euclidean separation이 작은 feature activation 벡터를 생성한다면,

신경망의 상위 레벨이 그것들을 유사하다고 생각할 수 있다.

테스트 세트의 5개 이미지와 각 이미지 세트와 가장 유사한 6개 이미지를 보여준다.

pixel 수준에서 검색된 학습 이미지는 일반적으로 L2에서 첫 번째 열의 쿼리 이미지와 가까이 있지 않다.

예를 들어, 검색된 개와 코끼리는 다양한 포즈로 나타난다.

4096 차원의 실수 벡터 두 개 사이의 유클리드 거리를 사용하여 유사도를 계산하는 것은 비효율적이지만,

이 벡터를 짧은 바이너리 코드로 압축하도록 autoencoder를 학습하면 효율적으로 만들 수 있다.

이것은 이미지 label 사용하지 않고 비슷한 의미의 패턴을 가진 이미지를 검색하는 경향이 있는 Autoencoder를 적용하는 것보다 훨씬 더 나은 이미지 검색 방법을 제공한다.

## Discussion

우리의 결과는 크고 깊은 CNN이 순전히 supervised learning을 사용하여 매우 어려운 데이터 set에서 기록적인 결과를 달성할 수 있음을 보여준다.

여기서, 단일 Conv-layer이 제거되면 우리 네트워크의 성능이 저하된다.

예를 들어, 중간 계층 중 하나를 제거하면 네트워크의 top-1 성능에 대해 약 2%의 손실이 발생한다.

따라서 깊이는 실제 error-rate와 연관이 있다.

이전처럼 unsupervised pre-training을 사용하지 않았다.

Labeled data 양을 증가시키지 않으면서도 네트워크의 크기를 크게 증가시킬 수있는 충분한 계산 능력을 얻었다.

네트워크가 크고 깊게 학습되었으므로 결과는 향상되었지만 인간의 시각체계의 지각-시간 경로와 일치시키기 위해서는 여전히 많은 시도가 필요하다.

궁극적으로 우리는 시간적 구조가 static 이미지에서 누락되거나 덜 명확한 정보를 제공하는 비디오 sequence에 매우 크고 깊은 CNN을 사용하고자 한다.

---
## CODE

In [1]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Contains a model definition for AlexNet.
This work was first described in:
  ImageNet Classification with Deep Convolutional Neural Networks
  Alex Krizhevsky, Ilya Sutskever and Geoffrey E. Hinton
and later refined in:
  One weird trick for parallelizing convolutional neural networks
  Alex Krizhevsky, 2014
Here we provide the implementation proposed in "One weird trick" and not
"ImageNet Classification", as per the paper, the LRN layers have been removed.
Usage:
  with slim.arg_scope(alexnet.alexnet_v2_arg_scope()):
    outputs, end_points = alexnet.alexnet_v2(inputs)
@@alexnet_v2
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

slim = tf.contrib.slim
trunc_normal = lambda stddev: tf.truncated_normal_initializer(0.0, stddev)


def alexnet_v2_arg_scope(weight_decay=0.0005):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                        activation_fn=tf.nn.relu, 
                        biases_initializer=tf.constant_initializer(0.1),
                        weights_regularizer=slim.l2_regularizer(weight_decay)):
        with slim.arg_scope([slim.conv2d], padding='SAME'):
            with slim.arg_scope([slim.max_pool2d], padding='VALID') as arg_sc:
                return arg_sc

def alexnet_v2(inputs,
               num_classes=1000,
               is_training=True,
               dropout_keep_prob=0.5,
               spatial_squeeze=True,
               scope='alexnet_v2'):
    """AlexNet version 2.
    Described in: http://arxiv.org/pdf/1404.5997v2.pdf
    Parameters from:
    github.com/akrizhevsky/cuda-convnet2/blob/master/layers/
    layers-imagenet-1gpu.cfg
    Note: All the fully_connected layers have been transformed to conv2d layers.
        To use in classification mode, resize input to 224x224. To use in fully
        convolutional mode, set spatial_squeeze to false.
        The LRN layers have been removed and change the initializers from
        random_normal_initializer to xavier_initializer.
    Args:
    inputs: a tensor of size [batch_size, height, width, channels].
    num_classes: number of predicted classes.
    is_training: whether or not the model is being trained.
    dropout_keep_prob: the probability that activations are kept in the dropout
      layers during training.
    spatial_squeeze: whether or not should squeeze the spatial dimensions of the
      outputs. Useful to remove unnecessary dimensions for classification.
    scope: Optional scope for the variables.
    Returns:
    the last op containing the log predictions and end_points dict.
    """
    with tf.variable_scope(scope, 'alexnet_v2', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'
        # Collect outputs for conv2d, fully_connected and max_pool2d.
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                            outputs_collections=[end_points_collection]):
            net = slim.conv2d(inputs, 64, [11, 11], 4, padding='VALID',
                            scope='conv1')
            net = slim.max_pool2d(net, [3, 3], 2, scope='pool1')
            net = slim.conv2d(net, 192, [5, 5], scope='conv2')
            net = slim.max_pool2d(net, [3, 3], 2, scope='pool2')
            net = slim.conv2d(net, 384, [3, 3], scope='conv3')
            net = slim.conv2d(net, 384, [3, 3], scope='conv4')
            net = slim.conv2d(net, 256, [3, 3], scope='conv5')
            net = slim.max_pool2d(net, [3, 3], 2, scope='pool5')

            # Use conv2d instead of fully_connected layers.
            with slim.arg_scope([slim.conv2d],
                              weights_initializer=trunc_normal(0.005),
                              biases_initializer=tf.constant_initializer(0.1)):
                net = slim.conv2d(net, 4096, [5, 5], padding='VALID',
                                  scope='fc6')
                net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                                   scope='dropout6')
                net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
                net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                                   scope='dropout7')
                net = slim.conv2d(net, num_classes, [1, 1],
                                  activation_fn=None,
                                  normalizer_fn=None,
                                  biases_initializer=tf.zeros_initializer(),
                                  scope='fc8')

            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            if spatial_squeeze:
                net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                end_points[sc.name + '/fc8'] = net
            return net, end_points
alexnet_v2.default_image_size = 224

#### Reference
- http://vision.stanford.edu/teaching/cs231b_spring1415/slides/alexnet_tugce_kyunghee.pdf
- http://laonple.blog.me/220654387455